In [1]:
from mpt.module.configuration_mpt import MPTConfig
from mpt.module.modeling_mpt import MPTForCausalLM, MPTModel, MPTBlock
from transformers import AutoTokenizer

In [2]:
config = MPTConfig()

In [3]:
config.attn_config["attn_impl"] = "torch"

In [4]:
config

MPTConfig {
  "attn_config": {
    "alibi": false,
    "alibi_bias_max": 8,
    "attn_impl": "torch",
    "attn_pdrop": 0.0,
    "attn_type": "multihead_attention",
    "attn_uses_sequence_id": false,
    "clip_qkv": null,
    "prefix_lm": false,
    "qk_ln": false,
    "softmax_scale": null
  },
  "d_model": 2048,
  "emb_pdrop": 0.0,
  "embedding_fraction": 1.0,
  "expansion_ratio": 4,
  "init_config": {
    "fan_mode": "fan_in",
    "init_nonlinearity": "relu",
    "name": "kaiming_normal_"
  },
  "init_device": "cpu",
  "learned_pos_emb": true,
  "logit_scale": null,
  "max_seq_len": 2048,
  "model_type": "mpt",
  "n_heads": 16,
  "n_layers": 24,
  "no_bias": false,
  "norm_type": "low_precision_layernorm",
  "resid_pdrop": 0.0,
  "transformers_version": "4.28.1",
  "use_cache": false,
  "verbose": 0,
  "vocab_size": 50368
}

In [5]:
model = MPTForCausalLM(config)

/home/ray/default/mpt_repo/mpt/module/attention.py:148: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


In [6]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

In [7]:
from fairscale.nn.checkpoint import checkpoint_wrapper

In [8]:
model = checkpoint_wrapper(model, offload_to_cpu=True)

In [9]:
inputs = tokenizer(["hello my name is kourosh? "], return_tensors="pt")
inputs

{'input_ids': tensor([[25521,   619,  1416,   310,   465,   276,   287,  1200,    32,   209]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
output = model.forward(**inputs)

/home/ray/anaconda3/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


tensor([[[ 0.1252,  1.4075, -1.8487,  ..., -0.5698,  1.6197,  1.9951],
         [ 0.3545,  1.5721, -1.5741,  ..., -0.7453,  1.2746,  1.9712],
         [ 1.0514,  1.0456, -2.1576,  ..., -0.6029,  1.8335,  2.0054],
         ...,
         [ 0.8154,  0.5779, -0.8116,  ..., -0.6054,  1.6809,  2.6405],
         [ 0.2411,  0.5816, -1.6374,  ..., -1.1922,  1.7143,  2.7007],
         [ 1.0074,  0.9751, -1.9049,  ..., -1.4873,  1.9630,  2.6677]]])


str